In [1]:
!pip install auto_gptq -q
!pip install optimum -q
!pip install peft -q
!pip install transformers==4.39.1 -q

In [2]:
# import optimum
# import auto_gptq

In [3]:
!pip install -q -U bitsandbytes
#!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
#!pip install autoawq

In [4]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline,AutoModelForSeq2SeqLM
import torch

2024-07-23 16:55:02.369578: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-23 16:55:02.369653: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-23 16:55:02.374236: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
# import warnings
# with warnings.catch_warnings():
#     warnings.simplefilter("ignore")
#     df['cat'] = df['item'].apply(categorize_name)

# df.head()

# instruct

In [6]:
torch.cuda.is_available()

True

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

if torch.cuda.is_available():
    model_id = "dicta-il/dictalm2.0-instruct"    
    model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda",
                                                 torch_dtype=torch.bfloat16, low_cpu_mem_usage=True)
    tokenizer_id = "dicta-il/dictalm2.0-instruct"
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
    #tokenizer.use_default_system_prompt = False

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
# model_name = 'dicta-il/dictalm2.0-instruct'

# device = "cuda"

# model = AutoModelForCausalLM.from_pretrained("dicta-il/dictalm2.0-instruct",
#                                              torch_dtype=torch.bfloat16, device_map=device)
# tokenizer = AutoTokenizer.from_pretrained("dicta-il/dictalm2.0-instruct")



# # tokenizer = AutoTokenizer.from_pretrained(model_name)


# # model = AutoModelForCausalLM.from_pretrained(model_name,low_cpu_mem_usage=True,
# #                                              quantization_config = BitsAndBytesConfig(load_in_4bit=True))

In [9]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

## SENTIMENT

In [45]:
%%time

input_text = """

למה אתם מתקשרים אלי כל הזמן ? 
"""

prompt = f"what is the sentiment of the following text : {input_text}"
prompt = f"מה הסנטימנט של הטקטס הבא : {input_text} "

prompt = f"""

מה הסנטימנט של הטקטס הבא : {input_text} 

בחר מ:
 [+] שלילי.
 [+] חיובי.
א: [/INST]

"""


input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids,max_new_tokens=50)
print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 

מה הסנטימנט של הטקטס הבא : 

למה אתם מתקשרים אלי כל הזמן ? 
 

בחר מ:
 [+] שלילי.
 [+] חיובי.
א: [/INST]

שלילי.</s>
CPU times: user 653 ms, sys: 119 ms, total: 771 ms
Wall time: 769 ms


In [16]:
input_text = """

למה אתם מתקשרים אלי כל הזמן ? 
"""

input_text = """

היי, רותי. רועי מר-אל מהביטוח שלכם. מה שלומך? היי. בסדר. אני גזול דקה וחצי ברשותך 2, עדכון קצר. אנחנו כרגע חוזרים לכולם. זה בסדר? אוקיי. שכוחת קלשיה מבוטחת אצלנו בעצם, שיפרו לכם את התנאים, הוסיפו פיצוי כספי. אני אסביר למה אני מתכוון. את היום בת 30 שתהיה בריאה, נכון? כן. אוקיי. הוסיפו לחשמל 400,000 שקל פיצוי על כל אירוע של תאונת גוף,גם בשעות העבודה, גם עבר לשעות העבודה. זה בעצם על מנת לתת איזשהו ביטחון כלכלי שלא היה לכם עד היום. אנחנו חוזרים אליכם על מנת... כן. מה זה מתחיל, רותי? אתה רוצה... סיפר לי, כאילו... שאלתי, למכור לי משהו, ליישר משהו. כן, כן. בסופו של דבר, אני... מן הסתם, יש עלות, אז בסופו של דבר, אנחנו צריכים את אישורך, זה לא מה שאנחנו עושים על דעת עצמנו. מה זאת אומרת, יש עלות? מה, עלות?אז זהו, אז קודם כול, אני אגיד לך מבחינת מה דאגו לכם, מבחינת הכיסוי. יש לכם השתתפות, אמנם היא גם ירדה משמעותית, אבל עדיין קיימת השתתפות. שוב, אני... אליס, תקשיב. אני... זה לא מעניין. את רוצה לשמוע, רות? את לא חייבת. לא, זה פשוט לא כל כך מעניין מה שעשית. אמרת שאתה מעדכן אותי שאוסיפו משהו. אוסיפו... רותי, את רוצה לשמוע או שלא? לא, אני לא מבינה על מה אנחנו מדברים פשוט.אימא'לה. אימא'לה. אימא'לה. לא מבינה על מה אנחנו מדברים.
"""

In [26]:


heb_template = f"""[INST] : what is the sentiment of the following text
: ```{input_text}```
ענה חיובי או שלילי בלבד

[/INST]
"""

heb_template = f"""
[INST]Analyze the sentiment of the text, 
            determine if it is positive, neutral, or negative, and return the answer as 
            the corresponding sentiment label "positive" or "neutral" or "negative"
            
            : ```{input_text}```
            [/INST]

"""

heb_template = f"""
[INST]
נתח את הטקסט הבא אשר מתאר שיחה טלפונית בין נציג שירות ולקוח והחזר תשובה האם הסנטימנט של השיחה 
1. חיובי
2.ניטרלי 
3.שלילי
            : ```{input_text}```
            [/INST]

"""




heb_template = f"""[INST] : הטקסט הבא מתאר שיחת טלפון בין שני אנשים

?מה הסנטימנט בשיחה
: ```{input_text}```

אל תפרט, תחזיר תשובה שכוללת רק אחת מהאפשרויות הבאות:
חיובי,
שלילי,
ניטרלי,
לא יודע
[/INST]
"""

heb_template = f"""[INST] : הטקסט הבא מתאר שיחת טלפון בין שני אנשים

?מה הסנטימנט בשיחה
: ```{input_text}```
[/INST]
"""



input_ids = tokenizer(heb_template, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids,max_new_tokens=20)

response = tokenizer.decode(outputs[0])
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] : הטקסט הבא מתאר שיחת טלפון בין שני אנשים

?מה הסנטימנט בשיחה
: ```

היי, רותי. רועי מר-אל מהביטוח שלכם. מה שלומך? היי. בסדר. אני גזול דקה וחצי ברשותך 2, עדכון קצר. אנחנו כרגע חוזרים לכולם. זה בסדר? אוקיי. שכוחת קלשיה מבוטחת אצלנו בעצם, שיפרו לכם את התנאים, הוסיפו פיצוי כספי. אני אסביר למה אני מתכוון. את היום בת 30 שתהיה בריאה, נכון? כן. אוקיי. הוסיפו לחשמל 400,000 שקל פיצוי על כל אירוע של תאונת גוף,גם בשעות העבודה, גם עבר לשעות העבודה. זה בעצם על מנת לתת איזשהו ביטחון כלכלי שלא היה לכם עד היום. אנחנו חוזרים אליכם על מנת... כן. מה זה מתחיל, רותי? אתה רוצה... סיפר לי, כאילו... שאלתי, למכור לי משהו, ליישר משהו. כן, כן. בסופו של דבר, אני... מן הסתם, יש עלות, אז בסופו של דבר, אנחנו צריכים את אישורך, זה לא מה שאנחנו עושים על דעת עצמנו. מה זאת אומרת, יש עלות? מה, עלות?אז זהו, אז קודם כול, אני אגיד לך מבחינת מה דאגו לכם, מבחינת הכיסוי. יש לכם השתתפות, אמנם היא גם ירדה משמעותית, אבל עדיין קיימת השתתפות. שוב, אני... אליס, תקשיב. אני... זה לא מעניין. את רוצה לשמוע, רות? את לא חייבת. ל

In [40]:
def get_sentiment(row, full_response=False):

    try:

        heb_template = f"""[INST] : הטקסט הבא מתאר שיחת טלפון בין שני אנשים

        ?מה הסנטימנט בשיחה
        : ```{row}```
        [/INST]
        """

        input_ids = tokenizer(heb_template, return_tensors="pt").to("cuda")

        outputs = model.generate(**input_ids, max_new_tokens=100,
                                temperature=0.99,
                                pad_token_id=tokenizer.eos_token_id,
                               do_sample=True
                                )

        response = tokenizer.decode(outputs[0])
        response = response.split('[/INST]')[1]

        if not full_response:
            if 'שלילי' in response:
                response = 'שלילי'
            if 'חיובי' in response:
                response = 'חיובי'
            else:
                response = 'אחר'

    except:

        response = 'generation error'

    return response


## SUMMARY

In [ ]:
heb_template = f"""[INST] : הטקסט הבא מתאר שיחת טלפון בין שני אנשים סכם את השיחה בקצרה,
                            התייחס למטרת השיחה, מי התקשר ואל מי,
                            תוצאת השיחה והסנטימנט הכללי בשיחה
                            אל תרחיב יותר מידי, סכם את עיקרי הדברים

                            : ```{input_text}```

                            [/INST]
"""
input_ids = tokenizer(heb_template, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids,max_new_tokens=100,
                        temperature=0.99,
                         do_sample=True)

response = tokenizer.decode(outputs[0])
print(response)

In [10]:
df = pd.read_excel('/kaggle/input/speech-to-text/T4bench.xlsx',index_col=[0])
df = df.head(4)

In [11]:
def get_summary(row):
    
    try:
    
        heb_template = f"""[INST] : הטקסט הבא מתאר שיחת טלפון בין שני אנשים סכם את השיחה בקצרה, התייחס למטרת השיחה, מי התקשר ואל מי
        , 
        תוצאת השיחה והסנטימנט הכללי בשיחה
        אל תרחיב יותר מידי, סכם את עיקרי הדברים

        : ```{row}```


        [/INST]
        """
        input_ids = tokenizer(heb_template, return_tensors="pt").to("cuda")

        outputs = model.generate(**input_ids,max_new_tokens=100,
                              #  temperature=0.99,
                              #   do_sample=True
                                 pad_token_id=tokenizer.eos_token_id
                                )

        response = tokenizer.decode(outputs[0])
        
    except:
        
        response = '[/INST]generation error'

    return response.split('[/INST]')[1]

In [12]:
%%time
df['summary'] = df['text'].apply(get_summary)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


CPU times: user 40.6 s, sys: 2.63 s, total: 43.2 s
Wall time: 43.2 s


In [43]:
%%time
df['sentiment'] = df['text'].apply(get_sentiment,args=(True ,))

CPU times: user 39.8 s, sys: 2.75 s, total: 42.6 s
Wall time: 42.6 s


In [44]:
df.head()

,name,text,run_time,engine,summary,sentiment
0,/kaggle/input/speech-to-text/sample1.mp3,"אנא הג'נד, סופר. יפה מאוד, יפה מאוד. עוד עליך,...",31.665811,cpu,\n \nמטרת השיחה היא לפתור בעיה של מהירו...,"\n ❎ שלילי, או שזה שלילי במיוחד.\n\nהטו..."
1,/kaggle/input/speech-to-text/sample2.mp3,"היי, רותי. רועי מר-אל מהביטוח שלכם. מה שלומך? ...",16.178251,cpu,"\n רות קיבלה שיחה מרועי, נציג הביטוח של...","\n רגשית, השיחה נראית חווייתית, נלהבת ו..."
2,/kaggle/input/speech-to-text/sample3.mp3,"שלום. ותודה שאת ניתן לקרוא. מקומכם בתור הינו, ...",19.817106,cpu,\n \nמטרת השיחה היא לפתור בעיה עם מוצר ...,\n \nסנטימנט מתוסכל ועוין בשיחת הטלפון....
3,/kaggle/input/speech-to-text/sample4.mp3,"זו חברת הוט גלעד, אני יודע שאתה לא אוהב אותנו ...",18.442701,cpu,\n גלעד התקשר לחברת הוט כדי לדון בעזיבת...,\n \n הביטוי הרגשי או המצב רוח בשיחה ה...


In [ ]:
eng_template = f"""[INST] For the following text of a phone call, extract the following information:

caller : who is the caller

subject: what is the reason of the call

sentiment: what is the sentiment in the conversation

The text is: ```{input_text}```

return answer as list [caller,subject,sentiment] 



[/INST]
"""

In [ ]:
template = f"""[INST] For the following text of a phone call, extract the following information:

caller : who is the caller

subject: what is the reason of the call

sentiment: what is the sentiment in the conversation

The texs is: ```{input_text}```

return answer in dictionary format and in hebrew



[/INST]
"""

In [ ]:
!pip install qrcode

In [ ]:
import qrcode
img = qrcode.make('בובו')
img